In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)


Mounted at /content/drive


In [ ]:
!ls "/content/"

drive  sample_data


In [ ]:
!unzip "/content/drive/My Drive/COL774 Assignment 4 Data/train_data.zip" -d "/content/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/train_data/res41812.jpg  
  inflating: /content/train_data/res16792.jpg  
  inflating: /content/train_data/res39771.jpg  
  inflating: /content/train_data/res48188.jpg  
  inflating: /content/train_data/res36442.jpg  
  inflating: /content/train_data/res49296.jpg  
  inflating: /content/train_data/res7697.jpg  
  inflating: /content/train_data/res20220.jpg  
  inflating: /content/train_data/res26889.jpg  
  inflating: /content/train_data/res30033.jpg  
  inflating: /content/train_data/res9862.jpg  
  inflating: /content/train_data/res31339.jpg  
  inflating: /content/train_data/res29576.jpg  
  inflating: /content/train_data/res26645.jpg  
  inflating: /content/train_data/res18973.jpg  
  inflating: /content/train_data/res37990.jpg  
  inflating: /content/train_data/res39765.jpg  
  inflating: /content/train_data/res16786.jpg  
  inflating: /content/train_data/res41806.jpg  
  inflating: /content/train_data/res364

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from skimage import io, transform

import matplotlib.pyplot as plt # for plotting
import numpy as np
import pandas as pd

In [ ]:
pd.read_csv("/content/drive/MyDrive/COL774 Assignment 4 Data/Train_text.tsv")


,train_data/res1.jpg\ttwo men in green shirts
0,train_data/res2.jpg\ta girl in an innertube
1,train_data/res3.jpg\tseveral men in hard hats are
2,train_data/res4.jpg\ta woman in a brown
3,train_data/res5.jpg\ta little girl in a
4,train_data/res6.jpg\tfive people are playing in a
...,...
49994,train_data/res49996.jpg\ttwo different kayakin...
49995,train_data/res49997.jpg\tthe victorian buildin...
49996,train_data/res49998.jpg\tthe two man are dancing
49997,train_data/res49999.jpg\ta man is leaning off


### Image Transforms

In [ ]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, image):
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)
        img = transform.resize(image, (new_h, new_w))
        return img


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, image):
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return image


IMAGE_RESIZE = (256, 256)
# Sequentially compose the transforms
img_transform = transforms.Compose([Rescale(IMAGE_RESIZE), ToTensor()])


### Captions Preprocessing

In [ ]:
# import nltk
# nltk.download('punkt')

class CaptionsPreprocessing:
    """Preprocess the captions, generate vocabulary and convert words to tensor tokens

    Args:
        captions_file_path (string): captions tsv file path
    """
    def __init__(self, captions_file_path):
        self.captions_file_path = captions_file_path

        # Read raw captions
        self.raw_captions_dict = self.read_raw_captions()

        # Preprocess captions
        self.captions_dict = self.process_captions()

        # Create vocabulary
        self.vocab = self.generate_vocabulary()

    def read_raw_captions(self):
        """
        Returns:
            Dictionary with raw captions list keyed by image ids (integers)
        """

        captions_dict = {}
        with open(self.captions_file_path, 'r', encoding='utf-8') as f:
            for img_caption_line in f.readlines():
                img_captions = img_caption_line.strip().split('\t')
                captions_dict[img_captions[0]] = img_captions[1]

        return captions_dict

    def process_captions(self):
        """
        Use this function to generate dictionary and other preprocessing on captions
        """

        raw_captions_dict = self.raw_captions_dict

        # Do the preprocessing here
        # IMPLEMENT THIS captions_dict = 
        captions_dict = {}
        captions_length = {}
        for key, caption in raw_captions_dict.items():
          enc = "<start> " + caption + " <end>"
          l1 = len(enc.split())
          for i in range(l1, 10):
            enc = enc + " <pad>"
          captions_dict[key] = enc

        return captions_dict

    def generate_vocabulary(self):
        """
        Use this function to generate dictionary and other preprocessing on captions
        """

        captions_dict = self.captions_dict

        # Generate the vocabulary
        # IMPLEMENT THIS vocab = 
        vocab = {"<start>": 0, "<end>": 1, "<pad>": 2, "<unk>": 3}
        i = 3
        for key, caption in captions_dict.items():
          words = caption.split()
          for word in words:
            if word not in vocab:
              vocab[word] = i
              i += 1     

        return vocab

    def captions_transform(self, img_caption_list):
        """
        Use this function to generate tensor tokens for the text captions
        Args:
            img_caption_list: List of captions for a particular image
        """
        vocab = self.vocab

        # Generate tensors
        # IMPLEMENT THIS
        
        words = img_caption_list.split()
        tokens = torch.zeros(10)
        l = len(words)
        for i in range(l):
          word = words[i]
          if word in vocab:
            tokens[i] = vocab[word] # <start>, <end> and <pad> tokens already included in each caption during preprocessing
          else:
            tokens[i] = 3 # vocab["<unk>"]
            
        return tokens

# Set the captions tsv file path
CAPTIONS_FILE_PATH = "/content/drive/MyDrive/COL774 Assignment 4 Data/Train_text.tsv"
captions_preprocessing_obj = CaptionsPreprocessing(CAPTIONS_FILE_PATH)

### Dataset Class

In [ ]:
class ImageCaptionsDataset(Dataset):

    def __init__(self, img_dir, captions_dict, img_transform=None, captions_transform=None):
        """
        Args:
            img_dir (string): Directory with all the images.
            captions_dict: Dictionary with captions list keyed by image paths (strings)
            img_transform (callable, optional): Optional transform to be applied
                on the image sample.

            captions_transform: (callable, optional): Optional transform to be applied
                on the caption sample (list).
        """
        self.img_dir = img_dir
        self.captions_dict = captions_dict
        self.img_transform = img_transform
        self.captions_transform = captions_transform

        self.image_ids = list(captions_dict.keys())

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = self.image_ids[idx]
        image = io.imread(img_name)
        if img_name in self.captions_dict:
            captions = self.captions_dict[img_name]
        else:
            captions = ""

        if self.img_transform:
            image = self.img_transform(image)

        if self.captions_transform:
            captions = self.captions_transform(captions)

        sample = {'image': image, 'captions': captions}

        return sample

### Model Architecture

In [ ]:
class Encoder(nn.Module):
    '''
      - encode images 
      - can use defined CNN architecture or ResNet50
    '''
    def __init__(self, pretrained):
        super(Encoder, self).__init__()
        self.pretrained = pretrained # False for self-defined architecture, True for ResNet50

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=4, stride=2) # 127 * 127
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # 62 * 62
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=3) # 20 * 20
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # 10 * 10
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=5, stride=1) # 6 * 6
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # 3 * 3
        self.fc1 = nn.Linear(in_features=16*3*3, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)

        # try and modify this
        premodel = torchvision.models.resnet50(pretrained = True)
        self.model = nn.Sequential(*list(premodel.children())[:-1]) # remove the last layer of ResNet50
        self.linear = nn.Linear(premodel.fc.in_features, 84) 
        self.bn = nn.BatchNorm1d(84, momentum=0.01)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, image_batch):

        if not self.pretrained:
            image_batch = self.pool(F.relu(self.conv1(image_batch)))
            image_batch = self.pool(F.relu(self.conv2(image_batch)))
            image_batch = self.pool(F.relu(self.conv3(image_batch)))
            image_features = torch.flatten(image_batch, 1) # flatten all dimensions except batch
            image_features = F.relu(self.fc1(image_features))
            image_features = F.relu(self.fc2(image_features))
            return image_features

        else:
            with torch.no_grad():
                image_features = self.model(image_batch)
            image_features = image_features.view(image_features.shape[0], -1)
            image_features = self.linear(image_features)
            image_features = self.relu(image_features)
            image_features = self.bn(image_features)
            return image_features

class Decoder(nn.Module):
    def __init__(self, vocab_size):
        super(Decoder, self).__init__()
        self.embed = nn.Embedding(num_embeddings=vocab_size, embedding_dim=84)
        self.lstm = nn.LSTM(input_size=84, hidden_size=168, num_layers=1, batch_first=True)
        self.linear = nn.Linear(in_features=168, out_features=vocab_size)
        self.dropout = nn.Dropout(p=0.5, inplace = True)   

    def forward(self, image_features, captions_batch):
        captions_batch = torch.tensor(captions_batch).to(torch.int64)
        captions_batch = self.embed(captions_batch)
        # print(image_features.shape, captions_batch.shape)
        embeddings = torch.cat((image_features.unsqueeze(1), captions_batch[:,:-1]), dim=1)
        embeddings = self.dropout(embeddings)
        # packed_seq = pack_padded_sequence(embeddings, lengths, batch_first=True, enforce_sorted= False)
        lstm_outputs, _ = self.lstm(embeddings)
        outputs = self.linear(lstm_outputs)
        return outputs

    def max_pred(self, image_features, hidden=None):
        # used in max-prediction
        output, hidden = self.lstm(image_features, hidden)
        output = self.linear(output.squeeze(1))
        return output, hidden


class ImageCaptionsNet(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(ImageCaptionsNet, self).__init__()

        self.pretrained = pretrained
        self.vocab_size = vocab_size

        self.encoder = Encoder(pretrained = self.pretrained)
        self.decoder = Decoder(vocab_size = self.vocab_size)     

    def forward(self, x):
        image_batch, captions_batch = x

        image_encodings = self.encoder(image_batch)
        # print(image_batch.shape)
        # print(image_encodings.shape)
        # print(captions_batch.shape)
        outputs = self.decoder(image_encodings, captions_batch)        
        # print(output.shape)

        return outputs

### Training Loop

In [ ]:
import time

# IMAGE_DIR = "/content/drive/MyDrive/COL774 Assignment 4 Data/train_data/"
IMAGE_DIR = "/content/train_data/"

vocab_size = len(captions_preprocessing_obj.vocab)

net = ImageCaptionsNet(False, vocab_size)
# If GPU training is required
net = net.cuda()

# Creating the Dataset
train_dataset = ImageCaptionsDataset(
    IMAGE_DIR, captions_preprocessing_obj.captions_dict, img_transform=img_transform,
    captions_transform=captions_preprocessing_obj.captions_transform
)

# Define your hyperparameters
NUMBER_OF_EPOCHS = 3
LEARNING_RATE = 1e-1
BATCH_SIZE = 32
NUM_WORKERS = 0 # Parallel threads for dataloading
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE)

restore = False
if restore == True:
    PATH = "/content/drive/MyDrive/COL774 Assignment 4 Data/checkpoints/latest_checkpoint.pth"
    net = ImageCaptionsNet()
    checkpoint = torch.load(PATH)
    net.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    iter = checkpoint['iter']
    loss = checkpoint['loss']
    net = net.cuda()

start_time = time.process_time()

# Creating the DataLoader for batching purposes
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
import os
for epoch in range(NUMBER_OF_EPOCHS):

    print("Epoch: " + str(epoch + 1))

    iteration = 0

    for batch_idx, sample in enumerate(train_loader):
        iteration += 1
        net.zero_grad()

        image_batch, captions_batch = sample['image'].float(), sample['captions']
        captions_batch = captions_batch.long() 

        # If GPU training required
        image_batch, captions_batch = image_batch.cuda(), captions_batch.cuda() # .float().cuda()

        output_captions = net((image_batch, captions_batch))
        loss = loss_function(output_captions.reshape(-1, output_captions.shape[2]), captions_batch.reshape(-1)) 
        loss.backward()
        optimizer.step()

        print(f"\tIteration: {iteration} Loss: {round(loss.item(), 5)} Time Elapsed: {round((time.process_time()-start_time)/60, 3)} min")

        if iteration % 25 == 0:
            PATH1 = "/content/drive/MyDrive/COL774 Assignment 4 Data/checkpoints/latest_iter.pth"
            torch.save({
                        'epoch': epoch,
                        'iter': iteration,
                        'model_state_dict': net.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': loss.item(),
                        }, PATH1)

    PATH2 = "/content/drive/MyDrive/COL774 Assignment 4 Data/checkpoints/latest_epoch.pth"
    torch.save({
                'epoch': epoch,
                'iter': iteration,
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss.item(),
                }, PATH2)

PATH3 = "/content/drive/MyDrive/COL774 Assignment 4 Data/checkpoints/final.pth"
torch.save({
          'epoch': NUMBER_OF_EPOCHS,
          'iter': 0,
          'model_state_dict': net.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': loss.item(),
          }, PATH3)        



Epoch: 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


	Iteration: 1 Loss: 8.95716 Time Elapsed: 0.032 min
	Iteration: 2 Loss: 8.94162 Time Elapsed: 0.066 min
	Iteration: 3 Loss: 8.91578 Time Elapsed: 0.098 min
	Iteration: 4 Loss: 8.89986 Time Elapsed: 0.13 min
	Iteration: 5 Loss: 8.87978 Time Elapsed: 0.163 min
	Iteration: 6 Loss: 8.86341 Time Elapsed: 0.196 min
	Iteration: 7 Loss: 8.84265 Time Elapsed: 0.229 min
	Iteration: 8 Loss: 8.81871 Time Elapsed: 0.261 min
	Iteration: 9 Loss: 8.78732 Time Elapsed: 0.294 min
	Iteration: 10 Loss: 8.77556 Time Elapsed: 0.325 min
	Iteration: 11 Loss: 8.75178 Time Elapsed: 0.357 min
	Iteration: 12 Loss: 8.71041 Time Elapsed: 0.39 min
	Iteration: 13 Loss: 8.70684 Time Elapsed: 0.424 min
	Iteration: 14 Loss: 8.65202 Time Elapsed: 0.457 min
	Iteration: 15 Loss: 8.62995 Time Elapsed: 0.489 min
	Iteration: 16 Loss: 8.65403 Time Elapsed: 0.522 min
	Iteration: 17 Loss: 8.60503 Time Elapsed: 0.554 min
	Iteration: 18 Loss: 8.53496 Time Elapsed: 0.586 min
	Iteration: 19 Loss: 8.58318 Time Elapsed: 0.618 min
	Ite

### Max Prediction

In [ ]:
vocab = captions_preprocessing_obj.vocab
index_to_word = {}
for word in vocab:
    index_to_word[vocab[word]] = word

def max_prediction(image_features, index_to_word, net):
    '''
       perform max prediction to generate captions (max length 10)
    '''
    caption_ids = []
    hidden = None
    x = image_features.unsqueeze(1)
    # print(x)
    with torch.no_grad():
        for i in range(10):            
            output, hidden = net.decoder.max_pred(x, hidden)
            # print(output, hidden)
            predicted_id = output.argmax(dim=1)
            x = net.decoder.embed(predicted_id).unsqueeze(1)            
            caption_ids.append(predicted_id)
    caption = [index_to_word[id.item()] for id in caption_ids]
    ignore = {"<start>", "<end>", "<pad>", "<unk>"}
    caption = [w for w in caption if w not in ignore]
    caption = " ".join(caption)
    
    return caption


In [ ]:
!unzip "/content/drive/My Drive/COL774 Assignment 4 Data/test_data.zip" -d "/content/"

In [ ]:
generated_captions = {}

TEST_DIR = "/content/test_data/"

test_dataset = ImageCaptionsDataset(
    TEST_DIR, captions_preprocessing_obj.captions_dict, img_transform=img_transform,
    captions_transform=captions_preprocessing_obj.captions_transform
)
test_dataset.image_ids = [f"test_data/test{i}.jpg" for i in range(1, 5001)]
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=NUM_WORKERS)

for batch_idx, sample in enumerate(test_loader):
    print(batch_idx, end='\t')
    image_batch = sample['image'].float()
    image_batch = image_batch.cuda()
    image_features = net.encoder(image_batch)
    predicted_caption = max_prediction(image_features, index_to_word, net)
    print(predicted_caption)
    generated_captions[batch_idx] = predicted_caption


0	a man in a blue
1	a man in a blue
2	a man in a blue
3	a man in a blue
4	a man in a blue
5	a man in a blue
6	a man in a blue
7	a man in a blue
8	a man in a blue
9	a man in a blue
10	a man in a blue
11	a man in a blue
12	a man in a blue
13	a man in a blue
14	a man in a blue
15	a man in a blue
16	a man in a blue
17	a man in a blue
18	a man in a blue
19	a man in a blue
20	a man in a blue
21	a man in a blue
22	a man in a blue
23	a man in a blue
24	a man in a blue
25	a man in a blue
26	a man in a blue
27	a man in a blue
28	a man in a blue
29	a man in a blue
30	a man in a blue
31	a man in a blue
32	a man in a blue
33	a man in a blue
34	a man in a blue
35	a man in a blue
36	a man in a blue
37	a man in a blue
38	a man in a blue
39	a man in a blue
40	a man in a blue
41	a man in a blue
42	a man in a blue
43	a man in a blue
44	a man in a blue
45	a man in a blue
46	a man in a blue
47	a man in a blue
48	a man in a blue
49	a man in a blue
50	a man in a blue
51	a man in a blue
52	a man in a blue
53	

In [ ]:
with open("you_know_who.tsv", "w") as f:
    for i in range(1, 5001):
        img_name = f"test_data/test{i}.jpg"
        caption = generated_captions[i-1]
        f.write(f"{img_name}\t{caption}\n")

In [ ]:
!cp "/content/you_know_who.tsv" "/content/drive/MyDrive/COL774 Assignment 4 Data/pred1.tsv"